In [2]:
import numpy as np 
import pandas as pd
import sweetviz as sv

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df_train=pd.read_csv("D:\\Foundathon\\Obesity prediction\\train.csv")
df_test=pd.read_csv("D:\\Foundathon\\Obesity prediction\\test.csv")
sample_submission=pd.read_csv("D:\\Foundathon\\Obesity prediction\\sample_submission.csv")
#report = sv.analyze(df_train)
df_train

c:\Users\grove\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



,id,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,Male,24.443011,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,2.763573,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II
1,1,Female,18.000000,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,2.000000,no,1.000000,1.000000,no,Automobile,Normal_Weight
2,2,Female,18.000000,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,1.910378,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight
3,3,Female,20.952737,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,1.674061,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III
4,4,Male,31.641081,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,1.979848,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,20753,Male,25.137087,1.766626,114.187096,yes,yes,2.919584,3.000000,Sometimes,no,2.151809,no,1.330519,0.196680,Sometimes,Public_Transportation,Obesity_Type_II
20754,20754,Male,18.000000,1.710000,50.000000,no,yes,3.000000,4.000000,Frequently,no,1.000000,no,2.000000,1.000000,Sometimes,Public_Transportation,Insufficient_Weight
20755,20755,Male,20.101026,1.819557,105.580491,yes,yes,2.407817,3.000000,Sometimes,no,2.000000,no,1.158040,1.198439,no,Public_Transportation,Obesity_Type_II
20756,20756,Male,33.852953,1.700000,83.520113,yes,yes,2.671238,1.971472,Sometimes,no,2.144838,no,0.000000,0.973834,no,Automobile,Overweight_Level_II


In [3]:
categorical_features = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']
numerical_features = [feature for feature in df_test.columns[1:] if feature not in categorical_features]

In [4]:
for feature in categorical_features:
    dummies = pd.get_dummies(df_test[feature], prefix=feature, dtype=int)
    df_test = pd.concat([dummies, df_test], axis=1)
    
    dummies = pd.get_dummies(df_train[feature], prefix=feature, dtype=int)
    df_train = pd.concat([dummies, df_train], axis=1)

df_test.drop(columns=categorical_features, inplace=True)
df_train.drop(columns=categorical_features, inplace=True)

In [5]:
# dummies = pd.get_dummies(df_train['NObeyesdad'], prefix='target', dtype=int)
# df_train = pd.concat([df_train, dummies], axis=1)
# df_train.drop(columns=['NObeyesdad'], inplace=True)

# target_columns = [elem for elem in df_train.columns if elem.startswith('target')]
target_columns = 'NObeyesdad'
df_train[target_columns].unique()

array(['Overweight_Level_II', 'Normal_Weight', 'Insufficient_Weight',
       'Obesity_Type_III', 'Obesity_Type_II', 'Overweight_Level_I',
       'Obesity_Type_I'], dtype=object)

In [6]:
def encodeTarget(obesity):
    dictObesity = {'Insufficient_Weight' : 0, 
                  'Normal_Weight' : 1,
                  'Overweight_Level_I': 2,
                  'Overweight_Level_II' : 3,
                  'Obesity_Type_I' : 4,
                  'Obesity_Type_II' : 5,
                  'Obesity_Type_III' : 6}
    return(dictObesity[obesity])

def unencodeTarget(key):
    revertedDictObesity = {0: 'Insufficient_Weight',
                           1: 'Normal_Weight',
                           2: 'Overweight_Level_I',
                           3: 'Overweight_Level_II',
                           4: 'Obesity_Type_I',
                           5: 'Obesity_Type_II',
                           6: 'Obesity_Type_III'}
    return(revertedDictObesity[key])

encodeTarget('Overweight_Level_II'), unencodeTarget(3)

(3, 'Overweight_Level_II')

In [7]:
df_train[target_columns] = df_train[target_columns].map(lambda x : encodeTarget(x))

In [8]:
print(df_train.columns)


Index(['MTRANS_Automobile', 'MTRANS_Bike', 'MTRANS_Motorbike',
       'MTRANS_Public_Transportation', 'MTRANS_Walking', 'CALC_Frequently',
       'CALC_Sometimes', 'CALC_no', 'SCC_no', 'SCC_yes', 'SMOKE_no',
       'SMOKE_yes', 'CAEC_Always', 'CAEC_Frequently', 'CAEC_Sometimes',
       'CAEC_no', 'FAVC_no', 'FAVC_yes', 'family_history_with_overweight_no',
       'family_history_with_overweight_yes', 'Gender_Female', 'Gender_Male',
       'id', 'Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE',
       'NObeyesdad'],
      dtype='object')


In [34]:
# column_to_drop = 'CALC_Always'
# if column_to_drop in df_test.columns:
#     df_test.drop(column_to_drop, axis=1, inplace=True)
#     print(f"Column '{column_to_drop}' successfully dropped.")
# else:
#     print(f"Column '{column_to_drop}' not found in the DataFrame.")


In [35]:
import imblearn
print(imblearn.__version__)

0.12.0


In [36]:
from imblearn.over_sampling import SMOTE

oversample = SMOTE()

X, y = df_train.drop([target_columns, 'id'], axis = 1), df_train[target_columns]
X, y = oversample.fit_resample(X, y)

df_train_resampled = pd.concat([X, y], axis = 1)


In [37]:
df_train_resampled

,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking,CALC_Frequently,CALC_Sometimes,CALC_no,SCC_no,SCC_yes,...,Gender_Male,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,NObeyesdad
0,0,0,0,1,0,0,1,0,1,0,...,1,24.443011,1.699998,81.669950,2.000000,2.983297,2.763573,0.000000,0.976473,3
1,1,0,0,0,0,0,0,1,1,0,...,0,18.000000,1.560000,57.000000,2.000000,3.000000,2.000000,1.000000,1.000000,1
2,0,0,0,1,0,0,0,1,1,0,...,0,18.000000,1.711460,50.165754,1.880534,1.411685,1.910378,0.866045,1.673584,0
3,0,0,0,1,0,0,1,0,1,0,...,0,20.952737,1.710730,131.274851,3.000000,3.000000,1.674061,1.467863,0.780199,6
4,0,0,0,1,0,0,1,0,1,0,...,1,31.641081,1.914186,93.798055,2.679664,1.971472,1.979848,1.967973,0.931721,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28317,1,0,0,0,0,0,1,0,1,0,...,1,29.009026,1.757554,118.204862,2.131753,3.000000,2.046959,0.344678,0.416458,5
28318,1,0,0,0,0,0,1,0,1,0,...,1,26.965544,1.772594,118.169420,2.966159,3.000000,2.096761,0.680259,0.192286,5
28319,1,0,0,0,0,0,1,0,1,0,...,1,27.266287,1.791737,120.151967,2.638510,3.000000,2.162104,1.277627,0.005161,5
28320,0,0,0,1,0,0,1,0,1,0,...,1,25.427505,1.763019,115.430080,2.043766,3.000000,2.151746,1.089455,1.054899,5


In [38]:
# import xgboost as xgb
# from bayes_opt import BayesianOptimization
# from sklearn.model_selection import cross_val_score
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split


# # Définir la fonction à optimiser
# def xgb_evaluate(max_depth, min_child_weight, gamma, subsample, colsample_bytree, 
#                  alpha, lambda_, learning_rate):
#     params = {'eval_metric': 'logloss',
#               'max_depth': int(max_depth),
#               'min_child_weight': min_child_weight,
#               'subsample': subsample,
#               'gamma': gamma,
#               'colsample_bytree': colsample_bytree,
#               'alpha': alpha,
#               'lambda': lambda_,
#               'learning_rate': learning_rate,
#               'use_label_encoder': False,
#               'objective': 'binary:logistic',
#               'seed': 42}
#     # Le modèle XGBClassifier est créé avec les paramètres sélectionnés
#     clf = xgb.XGBClassifier(**params)
#     # Calcul de la précision en utilisant la validation croisée
#     scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
#     return scores.mean()

# # Définir les bornes des paramètres à optimiser
# param_bounds = {
#     'max_depth': (3, 12),
#     'min_child_weight': (0, 10),
#     'gamma': (0, 5),
#     'subsample': (0.5, 1.0),
#     'colsample_bytree': (0.3, 0.9),
#     'alpha': (0, 10),
#     'lambda_': (1, 10),
#     'learning_rate': (0.01, 0.3)
# }

# # Utiliser BayesianOptimization pour optimiser les hyperparamètres
# optimizer = BayesianOptimization(
#     f=xgb_evaluate,
#     pbounds=param_bounds,
#     random_state=42,
# )

# # Démarrer l'optimisation
# optimizer.maximize(init_points=10, n_iter=600)

# # Afficher les meilleurs paramètres
# print(optimizer.max)



import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Assuming X and y are your features and labels
# Replace these with your actual data
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

# Define the function to optimize
def xgb_evaluate(max_depth, min_child_weight, gamma, subsample, colsample_bytree, 
                 alpha, lambda_, learning_rate):
    params = {'eval_metric': 'logloss',
              'max_depth': int(max_depth),
              'min_child_weight': min_child_weight,
              'subsample': subsample,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree,
              'alpha': alpha,
              'lambda': lambda_,
              'learning_rate': learning_rate,
              'use_label_encoder': False,
              'objective': 'binary:logistic',
              'seed': 42}
    clf = xgb.XGBClassifier(**params)
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    return scores.mean()

# Define the bounds for hyperparameters
param_bounds = {
    'max_depth': (3, 12),
    'min_child_weight': (0, 10),
    'gamma': (0, 5),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.3, 0.9),
    'alpha': (0, 10),
    'lambda_': (1, 10),
    'learning_rate': (0.01, 0.3)
}

# Use BayesianOptimization for hyperparameter optimization
optimizer = BayesianOptimization(
    f=xgb_evaluate,
    pbounds=param_bounds,
    random_state=42,
)

# Start the optimization with a reduced number of iterations
# Adjust the 'n_iter' value based on your desired runtime
optimizer.maximize(init_points=10, n_iter=50)

# Display the best hyperparameter values
print(optimizer.max)


|   iter    |  target   |   alpha   | colsam... |   gamma   |  lambda_  | learni... | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------------------------------------------
| 1         | 0.889     | 3.745     | 0.8704    | 3.66      | 6.388     | 0.05525   | 4.404     | 0.5808    | 0.9331    |
| 2         | 0.89      | 6.011     | 0.7248    | 0.1029    | 9.729     | 0.2514    | 4.911     | 1.818     | 0.5917    |
| 3         | 0.894     | 3.042     | 0.6149    | 2.16      | 3.621     | 0.1874    | 4.255     | 2.921     | 0.6832    |
| 4         | 0.892     | 4.561     | 0.7711    | 0.9984    | 5.628     | 0.1818    | 3.418     | 6.075     | 0.5853    |
| 5         | 0.888     | 0.6505    | 0.8693    | 4.828     | 8.276     | 0.09834   | 3.879     | 6.842     | 0.7201    |
| 6         | 0.9       | 1.22      | 0.5971    | 0.1719    | 9.184     | 0.08505   | 8.963     | 3.117     | 0.76      |
| 7         | 0.88      

In [41]:
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Assuming X and y are your features and labels
# Replace these with your actual data
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

# Define the function to optimize
def xgb_evaluate(max_depth, min_child_weight, gamma, subsample, colsample_bytree, 
                 alpha, lambda_, learning_rate):
    params = {'eval_metric': 'logloss',
              'max_depth': int(max_depth),
              'min_child_weight': min_child_weight,
              'subsample': subsample,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree,
              'alpha': alpha,
              'lambda': lambda_,
              'learning_rate': learning_rate,
              'use_label_encoder': False,
              'objective': 'binary:logistic',
              'seed': 42}
    clf = xgb.XGBClassifier(**params)
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    return scores.mean()

# Define the bounds for hyperparameters
param_bounds = {
    'max_depth': (3, 12),
    'min_child_weight': (0, 10),
    'gamma': (0, 5),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.3, 0.9),
    'alpha': (0, 10),
    'lambda_': (1, 10),
    'learning_rate': (0.01, 0.3)
}

# Use BayesianOptimization for hyperparameter optimization
optimizer = BayesianOptimization(
    f=xgb_evaluate,
    pbounds=param_bounds,
    random_state=42,
)

# Start the optimization with a reduced number of iterations
# Adjust the 'n_iter' value based on your desired runtime
optimizer.maximize(init_points=10, n_iter=50)

# Display the best hyperparameter values
print(optimizer.max)

# Assuming 'params' is defined with the best hyperparameters
params = optimizer.max['params']

# Convert 'max_depth' to integer
params['max_depth'] = int(params['max_depth'])

# Training XGBoost model
model = xgb.XGBClassifier(**params)
model.fit(X, y)


|   iter    |  target   |   alpha   | colsam... |   gamma   |  lambda_  | learni... | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------------------------------------------
| 1         | 0.889     | 3.745     | 0.8704    | 3.66      | 6.388     | 0.05525   | 4.404     | 0.5808    | 0.9331    |
| 2         | 0.89      | 6.011     | 0.7248    | 0.1029    | 9.729     | 0.2514    | 4.911     | 1.818     | 0.5917    |
| 3         | 0.894     | 3.042     | 0.6149    | 2.16      | 3.621     | 0.1874    | 4.255     | 2.921     | 0.6832    |
| 4         | 0.892     | 4.561     | 0.7711    | 0.9984    | 5.628     | 0.1818    | 3.418     | 6.075     | 0.5853    |
| 5         | 0.888     | 0.6505    | 0.8693    | 4.828     | 8.276     | 0.09834   | 3.879     | 6.842     | 0.7201    |
| 6         | 0.9       | 1.22      | 0.5971    | 0.1719    | 9.184     | 0.08505   | 8.963     | 3.117     | 0.76      |
| 7         | 0.88      

c:\Users\grove\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning:

[10:03:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "lambda_" } are not used.




XGBClassifier(alpha=1.453486554830357, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7388798187238547, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.0, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              lambda_=6.615961350957569, learning_rate=0.17798870950316995,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=4.236515605517397, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, ...)

In [43]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from bayes_opt import BayesianOptimization
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score

# Assuming X and y are your features and labels
# Replace these with your actual data
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the function to optimize
def xgb_evaluate(max_depth, min_child_weight, gamma, subsample, colsample_bytree, 
                 alpha, lambda_, learning_rate):
    params = {'eval_metric': 'logloss',
              'max_depth': int(max_depth),
              'min_child_weight': min_child_weight,
              'subsample': subsample,
              'gamma': gamma,
              'colsample_bytree': colsample_bytree,
              'alpha': alpha,
              'lambda': lambda_,
              'learning_rate': learning_rate,
              'use_label_encoder': False,
              'objective': 'binary:logistic',
              'seed': 42}
    clf = xgb.XGBClassifier(**params)
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='accuracy')
    return scores.mean()

# Define the bounds for hyperparameters
param_bounds = {
    'max_depth': (3, 12),
    'min_child_weight': (0, 10),
    'gamma': (0, 5),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.3, 0.9),
    'alpha': (0, 10),
    'lambda_': (1, 10),
    'learning_rate': (0.01, 0.3)
}

# Use BayesianOptimization for hyperparameter optimization
optimizer = BayesianOptimization(
    f=xgb_evaluate,
    pbounds=param_bounds,
    random_state=42,
)

# Start the optimization with a reduced number of iterations
# Adjust the 'n_iter' value based on your desired runtime
optimizer.maximize(init_points=10, n_iter=50)

# Display the best hyperparameter values
print(optimizer.max)

# Assuming 'params' is defined with the best hyperparameters
params = optimizer.max['params']
params['max_depth'] = int(params['max_depth'])

# Training XGBoost model
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pd.DataFrame(model.predict(X_test), columns=['NObeyesdad'])


|   iter    |  target   |   alpha   | colsam... |   gamma   |  lambda_  | learni... | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------------------------------------------------------
| 1         | 0.8863    | 3.745     | 0.8704    | 3.66      | 6.388     | 0.05525   | 4.404     | 0.5808    | 0.9331    |
| 2         | 0.8838    | 6.011     | 0.7248    | 0.1029    | 9.729     | 0.2514    | 4.911     | 1.818     | 0.5917    |
| 3         | 0.8975    | 3.042     | 0.6149    | 2.16      | 3.621     | 0.1874    | 4.255     | 2.921     | 0.6832    |
| 4         | 0.88      | 4.561     | 0.7711    | 0.9984    | 5.628     | 0.1818    | 3.418     | 6.075     | 0.5853    |
| 5         | 0.885     | 0.6505    | 0.8693    | 4.828     | 8.276     | 0.09834   | 3.879     | 6.842     | 0.7201    |
| 6         | 0.8938    | 1.22      | 0.5971    | 0.1719    | 9.184     | 0.08505   | 8.963     | 3.117     | 0.76      |
| 7         | 0.87      

c:\Users\grove\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning:

[10:05:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "lambda_" } are not used.




In [48]:
import pandas as pd
import xgboost as xgb

# 1. Check and display the columns of the training and test sets
print("Training Set Columns:", df_train.columns)
print("Test Set Columns:", df_test.columns)

# 2. Check the number of features in the training and test sets
print("Number of Features in Training Set:", len(df_train.columns))
print("Number of Features in Test Set:", len(df_test.columns))

# 3. Define and initialize the hyperparameters for XGBoost
params = {
    'max_depth': 7,  # Set an appropriate integer value for max_depth
    'learning_rate': 0.1,
    'n_estimators': 100,
    # Add other parameters as needed
}

# 4. Train the XGBoost model with the correct features
# Assuming X_train and y_train are NumPy arrays
# Convert them to DataFrames
X_train = pd.DataFrame(X_train, columns=df_train.columns)
df_test = pd.DataFrame(df_test, columns=df_train.columns)

# Ensure the features in training and test sets are aligned
common_features = set(X_train.columns) & set(df_test.columns)
X_train = X_train[common_features]
df_test = df_test[common_features]

# Train the model
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# 5. Make predictions on the test set
y_pred = pd.DataFrame(model.predict(df_test), columns=['NObeyesdad'])

# Create a DataFrame for submission
df_submission = pd.concat([df_test['id'], y_pred], axis=1)


Training Set Columns: Index(['MTRANS_Automobile', 'MTRANS_Bike', 'MTRANS_Motorbike',
       'MTRANS_Public_Transportation', 'MTRANS_Walking', 'CALC_Frequently',
       'CALC_Sometimes', 'CALC_no', 'SCC_no', 'SCC_yes', 'SMOKE_no',
       'SMOKE_yes', 'CAEC_Always', 'CAEC_Frequently', 'CAEC_Sometimes',
       'CAEC_no', 'FAVC_no', 'FAVC_yes', 'family_history_with_overweight_no',
       'family_history_with_overweight_yes', 'Gender_Female', 'Gender_Male',
       'id', 'Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE',
       'NObeyesdad'],
      dtype='object')
Test Set Columns: Index(['MTRANS_Automobile', 'MTRANS_Bike', 'MTRANS_Motorbike',
       'MTRANS_Public_Transportation', 'MTRANS_Walking', 'CALC_Frequently',
       'CALC_Sometimes', 'CALC_no', 'SCC_no', 'SCC_yes', 'SMOKE_no',
       'SMOKE_yes', 'CAEC_Always', 'CAEC_Frequently', 'CAEC_Sometimes',
       'CAEC_no', 'FAVC_no', 'FAVC_yes', 'family_history_with_overweight_no',
       'family_history_with_overweight_yes', '

ValueError: Shape of passed values is (800, 20), indices imply (800, 32)

In [27]:
params_opti = {'eval_metric': 'logloss',
              'max_depth': int(12.0),
              'min_child_weight': 0.4004,
              'subsample': 1.0,
              'gamma': 0,
              'colsample_bytree': 0.9,
              'alpha': 0.112,
              'lambda': 2.653,
              'learning_rate': 0.3,
              'use_label_encoder': False,
              'objective': 'binary:logistic',
              'seed': 42}

model = xgb.XGBClassifier(**params_opti)
model.fit(X, y)

XGBClassifier(alpha=0.112, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=0, grow_policy=None,
              importance_type=None, interaction_constraints=None, lambda=2.653,
              learning_rate=0.3, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
              max_leaves=None, min_child_weight=0.4004, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, ...)

In [31]:
y_pred = pd.DataFrame(model.predict(df_test.drop('id', axis=1)), columns = ['NObeyesdad'])

df_submission = pd.concat([df_test['id'], y_pred], axis = 1)
df_submission

ValueError: Feature shape mismatch, expected: 20, got 30

In [ ]:
df_submission['NObeyesdad'] = df_submission['NObeyesdad'].map(lambda x : unencodeTarget(x))

In [ ]:
df_submission

In [ ]:
df_submission.to_csv('submissionXgb.csv', index = False)

In [ ]:
df_submission.loc[df_submission['NObeyesdad'] != 1]

In [32]:
# Training XGBoost model
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)


NameError: name 'params' is not defined